In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from IPython.display import HTML
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display
from geopy.distance import geodesic
from pytz import timezone


## Load crime data & slice for the school year 2022-2023

In [39]:
df_crimes_2023 = pd.read_csv('/Users/Pierre-Alexandre/Downloads/df_crimes_2023.csv')
df_crimes_2023.head()
df_crimes_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30336 entries, 0 to 30335
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INCIDENT_NUMBER      30336 non-null  object 
 1   OFFENSE_CODE         30336 non-null  int64  
 2   OFFENSE_CODE_GROUP   0 non-null      float64
 3   OFFENSE_DESCRIPTION  30336 non-null  object 
 4   DISTRICT             30243 non-null  object 
 5   REPORTING_AREA       30336 non-null  object 
 6   SHOOTING             30336 non-null  int64  
 7   OCCURRED_ON_DATE     30336 non-null  object 
 8   YEAR                 30336 non-null  int64  
 9   MONTH                30336 non-null  int64  
 10  DAY_OF_WEEK          30336 non-null  object 
 11  HOUR                 30336 non-null  int64  
 12  UCR_PART             0 non-null      float64
 13  STREET               30336 non-null  object 
 14  Lat                  26896 non-null  float64
 15  Long                 26896 non-null 

In [40]:
df_crimes2022 = pd.read_csv('/Users/Pierre-Alexandre/Downloads/Crimes_report_2022.csv')
df_crimes2022.head()
df_crimes2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73852 entries, 0 to 73851
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INCIDENT_NUMBER      73852 non-null  object 
 1   OFFENSE_CODE         73852 non-null  int64  
 2   OFFENSE_CODE_GROUP   0 non-null      float64
 3   OFFENSE_DESCRIPTION  73852 non-null  object 
 4   DISTRICT             73681 non-null  object 
 5   REPORTING_AREA       73852 non-null  object 
 6   SHOOTING             73852 non-null  int64  
 7   OCCURRED_ON_DATE     73852 non-null  object 
 8   YEAR                 73852 non-null  int64  
 9   MONTH                73852 non-null  int64  
 10  DAY_OF_WEEK          73852 non-null  object 
 11  HOUR                 73852 non-null  int64  
 12  UCR_PART             0 non-null      float64
 13  STREET               73851 non-null  object 
 14  Lat                  70044 non-null  float64
 15  Long                 70044 non-null 

/Users/Pierre-Alexandre/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
crimes_2022_2023_merged = pd.concat([df_crimes2022,df_crimes_2023])

In [42]:
crimes_2022_2023_merged.drop(['OFFENSE_CODE_GROUP','OFFENSE_CODE','UCR_PART'],axis = 'columns',inplace = True)

In [43]:
crimes_2022_2023_merged['OCCURRED_ON_DATE'] = pd.to_datetime(crimes_2022_2023_merged['OCCURRED_ON_DATE'], utc=True).dt.tz_convert('America/New_York')

In [44]:
crimes_2022_2023_filtered = crimes_2022_2023_merged[crimes_2022_2023_merged['OCCURRED_ON_DATE'] >= pd.to_datetime('2022-09-08 00:00:00', utc=True).tz_convert('America/New_York')]

## Keep only crimes that involved shooting to create the map for 2022-2023

In [48]:
crimes_2022_2023_filtered = crimes_2022_2023_filtered[crimes_2022_2023_filtered['SHOOTING'] == 1]

In [49]:
crimes_2022_2023_filtered.head()

,INCIDENT_NUMBER,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location
50986,222068976,INVESTIGATE PERSON,D14,,1,2022-09-08 09:04:00-04:00,2022,9,Thursday,13,COMMONWEALTH AVE,42.349066,-71.134147,"(42.34906577769542, -71.13414695373906)"
51068,222069135,INVESTIGATE PROPERTY,C11,250,1,2022-09-08 16:54:00-04:00,2022,9,Thursday,20,SAVIN HILL AVE,42.312197,-71.052048,"(42.312197463106756, -71.05204832519735)"
51151,222069199,BALLISTICS EVIDENCE/FOUND,A1,,1,2022-09-09 02:34:00-04:00,2022,9,Friday,6,NEW CHARDON ST,42.362607,-71.061327,"(42.3626074984039, -71.06132725274803)"
51301,222069437,BALLISTICS EVIDENCE/FOUND,B2,,1,2022-09-09 17:24:00-04:00,2022,9,Friday,21,DUDLEY ST,42.321970,-71.072425,"(42.32196970405995, -71.07242528935052)"
51524,222069744,INVESTIGATE PROPERTY,B3,,1,2022-09-10 19:05:00-04:00,2022,9,Saturday,23,RIPLEY RD,42.299263,-71.076265,"(42.2992630326008, -71.0762651335127)"


In [50]:
crimes_2022_2023_filtered.dropna(subset=['Location'], inplace=True)

In [51]:
crimes_2022_2023_filtered['Location'].isna().sum()

0

## Do the same for school year 2021-2022

In [55]:
df_crimes2021 = pd.read_csv('/Users/Pierre-Alexandre/Downloads/Crimes_report_2021.csv')
df_crimes2021.head()

/Users/Pierre-Alexandre/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,212021001,619,NaN,LARCENY ALL OTHERS,B2,181,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,ALLERTON ST & MAGAZINE ST\nROXBURY MA 02119\n...,42.327551,-71.069162,"(42.327550999587764, -71.0691619893362)"
1,212025963,3201,NaN,PROPERTY - LOST/ MISSING,B3,465,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,WHITFIELD ST,42.284826,-71.091374,"(42.28482576580488, -71.09137368938802)"
2,212022379,1102,NaN,FRAUD - FALSE PRETENSE / SCHEME,C6,574,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,MARINE RD,42.309719,-71.104294,"(42.30971856767274, -71.10429431787648)"
3,212019050,615,NaN,LARCENY THEFT OF MV PARTS & ACCESSORIES,A1,77,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,STANIFORD ST,42.361839,-71.059765,"(42.361838566564714, -71.05976489094158)"
4,212019422,3115,NaN,INVESTIGATE PERSON,B3,355,0,2021-01-01 00:00:00,2021,1,Friday,0,NaN,FULLER ST,42.297555,-71.059709,"(42.29755532959655, -71.05970910242573)"


In [56]:
df_crimes2021_2022_merged = pd.concat([df_crimes2021,df_crimes2022])

In [57]:
df_crimes2021_2022_merged.drop(['OFFENSE_CODE_GROUP','OFFENSE_CODE','UCR_PART'],axis = 'columns',inplace = True)

In [58]:
df_crimes2021_2022_merged['OCCURRED_ON_DATE'] = pd.to_datetime(df_crimes2021_2022_merged['OCCURRED_ON_DATE'], utc=True).dt.tz_convert('America/New_York')

In [59]:
crimes_2021_2022_filtered = df_crimes2021_2022_merged[(df_crimes2021_2022_merged['OCCURRED_ON_DATE'] >= pd.to_datetime('2021-09-09 00:00:00', utc=True).tz_convert('America/New_York')) &
                                                      (df_crimes2021_2022_merged['OCCURRED_ON_DATE'] <= pd.to_datetime('2022-06-22 00:00:00', utc=True).tz_convert('America/New_York'))]

## Keep only crimes that involved shooting to create the map for 2022-2023

In [63]:
crimes_2021_2022_filtered = crimes_2021_2022_filtered[crimes_2021_2022_filtered['SHOOTING'] == 1]

In [66]:
crimes_2021_2022_filtered.dropna(subset=['Location'], inplace=True)

/Users/Pierre-Alexandre/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [67]:
crimes_2021_2022_filtered['Location'].isna().sum()

0

## Load School location data

In [52]:
boston_schools = pd.read_excel('/Users/Pierre-Alexandre/Documents/documents/Intership_city_hall/BPS/BPS_geolocation.xlsx')
boston_schools

,name,adress,latitude,longitude
0,Young Achievers Science & Math K-8,"20 Outlook Rd, Mattapan, MA, 02126",42.282320,-71.095000
1,"Winthrop, John Elementary","35 Brookford St, Dorchester, MA, 02125",42.318210,-71.075630
2,"Winship, F. Lyman Elementary","54 Dighton St, Brighton, MA, 02135",42.347610,-71.155350
3,West Zone Early Learning Center,"200 Heath St., Jamaica Plain, MA, 02130",42.325727,-71.106434
4,William E Carter School,"396 Northampton St Boston, MA 02118",42.326340,-71.106850
...,...,...,...,...
107,"Beethoven, Elementary School","5125 Washington St., West Roxbury, MA , 02132",42.263524,-71.155792
108,Baldwin Early Learning Pilot Academy,"121 Corey Rd., Brighton, MA , 02135",42.342060,-71.140472
109,Another Course to College,"612 Metropolitan Ave, Hyde Park, MA , 02136",42.265060,-71.117790
110,Alighier School,"37 Gove Street, East Boston, MA , 02128",42.371429,-71.037697


## Create the Map for 2022-2023

In [54]:
# Get shooting coordinates
shooting_latitude = crimes_2022_2023_filtered['Lat'].tolist()
shooting_longitude = crimes_2022_2023_filtered['Long'].tolist()
coords1 = list(zip(shooting_latitude, shooting_longitude))

# Create map
map = folium.Map(location=coords1[0], zoom_start=14, tiles="OpenStreetMap")

# Marker cluster for shooting locations
mc_shootings = MarkerCluster(name='Crimes')

# Create a list to store the crimes within the radius of schools
crimes_within_radius = []

# Create a dictionary to store the schools where crimes occurred within the radius
schools_within_radius = {}

# Add shooting markers to the map and check for crimes within the radius of schools
for coord, offense_description, occurred_date in zip(coords1, crimes_2022_2023_filtered['OFFENSE_DESCRIPTION'], crimes_2022_2023_filtered['OCCURRED_ON_DATE']):
    popup_description = f"<b>Description:</b> {offense_description}"
    popup_date = f"<b>Occurred On Date:</b> {occurred_date}"
    popup_content = f"{popup_description}<br>{popup_date}"
    marker = folium.Marker(location=coord, popup=popup_content,
                           icon=folium.Icon(color='red', icon='exclamation-circle', prefix='fa'))
    mc_shootings.add_child(marker)
    
    # Check for crimes within the radius of schools
    for index, row in boston_schools.iterrows():
        school_location = (row['latitude'], row['longitude'])
        radius = 200  # Radius in meters
        
        # Calculate the distance between the shooting location and the school location
        distance = geodesic(coord, school_location).meters
        
        # If the distance is within the radius, add the crime to the list and store the school name
        if distance <= radius:
            crimes_within_radius.append(offense_description)
            if offense_description not in schools_within_radius:
                schools_within_radius[offense_description] = [row['name']]
            else:
                schools_within_radius[offense_description].append(row['name'])

mc_shootings.add_to(map)

# Create a feature group for the school locations
school_locations = folium.FeatureGroup(name='School Locations')

# Style the school markers and circles
for index, row in boston_schools.iterrows():
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(school_locations)
        
    folium.Circle(
        radius=200,
        location=[row['latitude'], row['longitude']],
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.4,
    ).add_to(school_locations)

# Add the school locations to the map
school_locations.add_to(map)

# Customize map appearance
folium.LayerControl().add_to(map)

# Create a label widget to display the number of crimes and crimes within the radius of schools
crimes_label = widgets.Label(value='Number of Crimes: 0')
crimes_within_radius_label = widgets.Label(value='Crimes within Radius: 0')
schools_within_radius_label = widgets.Label(value='Schools where Crimes Occurred within Radius:')

# Function to update the number of crimes and crimes within the radius based on the selected crime from the dropdown
def update_crimes_label(change):
    selected_crime = change['new']
    num_crimes = len(crimes_2022_2023_filtered[crimes_2022_2023_filtered['OFFENSE_DESCRIPTION'] == selected_crime])
    crimes_label.value = f'Number of Crimes: {num_crimes}'
    
    num_crimes_within_radius = crimes_within_radius.count(selected_crime)
    crimes_within_radius_label.value = f'Crimes within Radius: {num_crimes_within_radius}'
    
    if selected_crime in schools_within_radius:
        schools = schools_within_radius[selected_crime]
        schools_list = "\n".join(f"<li>{school.strip()}</li>" for school in schools)
        schools_within_radius_label.value = f'Schools where Crimes Occurred within Radius:<br><ul>{schools_list}</ul>'
    else:
        schools_within_radius_label.value = 'Schools where Crimes Occurred within Radius:'

# Create a dropdown widget with all different crimes
crime_dropdown = widgets.Dropdown(
    options=crimes_2022_2023_filtered['OFFENSE_DESCRIPTION'].unique(),
    description='Crime:'
)

# Create an HTML widget to display the schools within radius
schools_within_radius_label = widgets.HTML(value='Schools where Crimes Occurred within Radius:')

# Observe the change in the dropdown value and update the number of crimes and crimes within the radius
crime_dropdown.observe(update_crimes_label, 'value')

# Display the map and widgets
display(map)
display(widgets.VBox([crime_dropdown, crimes_label, crimes_within_radius_label, schools_within_radius_label]))


## Create the map for 2021-2022

In [68]:
# Get shooting coordinates
shooting_latitude = crimes_2021_2022_filtered['Lat'].tolist()
shooting_longitude = crimes_2021_2022_filtered['Long'].tolist()
coords1 = list(zip(shooting_latitude, shooting_longitude))

# Create map
map = folium.Map(location=coords1[0], zoom_start=14, tiles="OpenStreetMap")

# Marker cluster for shooting locations
mc_shootings = MarkerCluster(name='Crimes')

# Create a list to store the crimes within the radius of schools
crimes_within_radius = []

# Create a dictionary to store the schools where crimes occurred within the radius
schools_within_radius = {}

# Add shooting markers to the map and check for crimes within the radius of schools
for coord, offense_description, occurred_date in zip(coords1, crimes_2021_2022_filtered['OFFENSE_DESCRIPTION'], crimes_2021_2022_filtered['OCCURRED_ON_DATE']):
    popup_description = f"<b>Description:</b> {offense_description}"
    popup_date = f"<b>Occurred On Date:</b> {occurred_date}"
    popup_content = f"{popup_description}<br>{popup_date}"
    marker = folium.Marker(location=coord, popup=popup_content,
                           icon=folium.Icon(color='red', icon='exclamation-circle', prefix='fa'))
    mc_shootings.add_child(marker)
    
    # Check for crimes within the radius of schools
    for index, row in boston_schools.iterrows():
        school_location = (row['latitude'], row['longitude'])
        radius = 200  # Radius in meters
        
        # Calculate the distance between the shooting location and the school location
        distance = geodesic(coord, school_location).meters
        
        # If the distance is within the radius, add the crime to the list and store the school name
        if distance <= radius:
            crimes_within_radius.append(offense_description)
            if offense_description not in schools_within_radius:
                schools_within_radius[offense_description] = [row['name']]
            else:
                schools_within_radius[offense_description].append(row['name'])

mc_shootings.add_to(map)

# Create a feature group for the school locations
school_locations = folium.FeatureGroup(name='School Locations')

# Style the school markers and circles
for index, row in boston_schools.iterrows():
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(school_locations)
        
    folium.Circle(
        radius=200,
        location=[row['latitude'], row['longitude']],
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.4,
    ).add_to(school_locations)

# Add the school locations to the map
school_locations.add_to(map)

# Customize map appearance
folium.LayerControl().add_to(map)

# Create a label widget to display the number of crimes and crimes within the radius of schools
crimes_label = widgets.Label(value='Number of Crimes: 0')
crimes_within_radius_label = widgets.Label(value='Crimes within Radius: 0')
schools_within_radius_label = widgets.Label(value='Schools where Crimes Occurred within Radius:')

# Function to update the number of crimes and crimes within the radius based on the selected crime from the dropdown
def update_crimes_label(change):
    selected_crime = change['new']
    num_crimes = len(crimes_2021_2022_filtered[crimes_2021_2022_filtered['OFFENSE_DESCRIPTION'] == selected_crime])
    crimes_label.value = f'Number of Crimes: {num_crimes}'
    
    num_crimes_within_radius = crimes_within_radius.count(selected_crime)
    crimes_within_radius_label.value = f'Crimes within Radius: {num_crimes_within_radius}'
    
    if selected_crime in schools_within_radius:
        schools = schools_within_radius[selected_crime]
        schools_list = "\n".join(f"<li>{school.strip()}</li>" for school in schools)
        schools_within_radius_label.value = f'Schools where Crimes Occurred within Radius:<br><ul>{schools_list}</ul>'
    else:
        schools_within_radius_label.value = 'Schools where Crimes Occurred within Radius:'

# Create a dropdown widget with all different crimes
crime_dropdown = widgets.Dropdown(
    options=crimes_2021_2022_filtered['OFFENSE_DESCRIPTION'].unique(),
    description='Crime:'
)

# Create an HTML widget to display the schools within radius
schools_within_radius_label = widgets.HTML(value='Schools where Crimes Occurred within Radius:')

# Observe the change in the dropdown value and update the number of crimes and crimes within the radius
crime_dropdown.observe(update_crimes_label, 'value')

# Display the map and widgets
display(map)
display(widgets.VBox([crime_dropdown, crimes_label, crimes_within_radius_label, schools_within_radius_label]))
